# Mlb Player_info

## imports 

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import sys

In [2]:
db_path = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/mlb_players.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Set up your DB path
base_dir = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr'
db_path = os.path.join(base_dir, 'data', 'mlb_players.db')  
pd.set_option('display.max_columns', None)

In [11]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn) 
print(tables) 

               name
0   mlb_player_info
1   sqlite_sequence
2  mlb_player_stats


## MLB Player info 

In [4]:
mlb_player_info = pd.read_sql_query("SELECT * FROM mlb_player_info", conn)

mlb_player_info.head()


,id,mlb_player_id,fullName,firstName,lastName,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,height,weight,primaryPosition,batSide,pitchHand,debutDate,active
0,1,671096,Andrew Abbott,Andrew,Abbott,1999-06-01,25,Lynchburg,VA,USA,"6' 0""",192,Pitcher,Left,Left,2023-06-05,1
1,2,682928,CJ Abrams,Paul,Abrams,2000-10-03,24,Alpharetta,GA,USA,"6' 0""",191,Shortstop,Left,Right,2022-04-08,1
2,3,650556,Bryan Abreu,Bryan,Abreu,1997-04-22,28,Santo Domingo Centro,None,Dominican Republic,"6' 1""",230,Pitcher,Right,Right,2019-07-31,1
3,4,677800,Wilyer Abreu,Wilyer,Abreu,1999-06-24,25,Maracaibo,None,Venezuela,"5' 10""",215,Outfielder,Left,Left,2023-08-22,1
4,5,682668,Luisangel Acuña,Luisangel,Acuña,2002-03-12,23,Caracas,None,Venezuela,"5' 8""",181,Second Base,Right,Right,2024-09-14,1


## MLB Player stats 

In [3]:
mlb_player_stats = pd.read_sql_query("SELECT * FROM mlb_player_stats", conn)

mlb_player_stats.head(10)

,mlb_player_id,stat_type,stat_group,season,game_date,team_id,team_name,opponent_id,opponent_name,position,gamesPlayed,games,gamesStarted,assists,putOuts,errors,chances,fielding,doublePlays,triplePlays,throwingErrors,rangeFactorPerGame,rangeFactorPer9Inn,innings,inningsPitched,catcherERA,flyOuts,groundOuts,airOuts,passedBall,wins,losses,wildPitches,pickoffs,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,avg,atBats,obp,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,groundIntoTriplePlay,numberOfPitches,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,summary,isHome,isWin,game_id,game_number,day_night,era,game
0,434378,season,fielding,2024,None,117.0,Houston Astros,NaN,None,NA,17,17,17.0,5.0,4.0,0.0,9.0,1.000,0.0,0.0,0.0,0.53,0.90,90.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
1,445276,season,hitting,2024,None,111.0,Boston Red Sox,NaN,None,None,54,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,0.0,0.0,NaN,None,None,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,0.0,.000,.000,.000,0.0,0.0,.---,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.---,-.--,0.0,-.--,None,NaN,NaN,NaN,NaN,None,na,na
2,445276,season,fielding,2024,None,111.0,Boston Red Sox,NaN,None,NA,54,54,0.0,1.0,1.0,0.0,2.0,1.000,0.0,0.0,0.0,0.04,0.33,54.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
3,445926,season,fielding,2024,None,144.0,Atlanta Braves,NaN,None,NA,46,46,0.0,8.0,2.0,1.0,11.0,.909,0.0,0.0,1.0,0.22,1.43,63.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
4,450203,season,fielding,2024,None,144.0,Atlanta Braves,NaN,None,NA,30,30,30.0,21.0,9.0,3.0,33.0,.909,0.0,0.0,3.0,1.00,1.64,165.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
5,453286,season,fielding,2024,None,140.0,Texas Rangers,NaN,None,NA,9,9,9.0,5.0,4.0,1.0,10.0,.900,0.0,0.0,1.0,1.00,1.88,43.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
6,455117,season,hitting,2024,None,145.0,Chicago White Sox,NaN,None,None,48,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,30.0,41.0,NaN,None,None,NaN,NaN,9.0,3.0,0.0,4.0,51.0,8.0,0.0,16.0,1.0,.119,135.0,.174,.230,.404,0.0,0.0,.---,5.0,NaN,556.0,147.0,31.0,11.0,56.0,3.0,0.0,.150,0.73,0.0,33.75,None,NaN,NaN,NaN,NaN,None,na,na
7,455117,season,fielding,2024,None,145.0,Chicago White Sox,NaN,None,NA,48,48,46.0,11.0,378.0,2.0,391.0,.995,0.0,0.0,1.0,8.10,9.02,388.2,None,4.61,NaN,NaN,NaN,3.0,None,None,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,3.0,39.0,.929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,1.0,None,None,NaN,NaN,NaN,NaN,None,na,na
8,455117,career,hitting,None,None,NaN,None,NaN,None,None,1191,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,897.0,887.0,NaN,None,None,NaN,NaN,361.0,133.0,4.0,117.0,1070.0,299.0,17.0,713.0,67.0,.203,3511.0,.278,.343,.621,5.0,3.0,.375,88.0,NaN,15313.0,3933.0,1205.0,377.0,1567.0,46.0,10.0,.255,1.01,0.0,30.01,None,NaN,NaN,NaN,NaN,None,na,na
9,455117,career,fielding,None,None,NaN,None,NaN,None,NA,1132,1132,1053.0,513.0,9044.0,63.0,9620.0,.993,47.0,0.0,51.0,8.44,9.32,9227.1,None,3.81,NaN,NaN,NaN,78.0,None,None,367.0,22.0,NaN,NaN,NaN,NaN,NaN,

## Player info and stats

In [7]:
query = """
SELECT mlb_player_stats.*, mlb_player_info.*
FROM mlb_player_stats
JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
"""
player_pre = pd.read_sql_query(query, conn)

# Reorder columns: fullName and mlb_player_id first, then the rest
cols = ['fullName', 'mlb_player_id'] + [col for col in player_pre.columns if col not in ['fullName', 'mlb_player_id']]
player_pre = player_pre[cols]

# Rename columns to make them more readable
player_df = player_pre.rename(columns={
    'fullName': 'Name',
    'mlb_player_id': 'PlayerID',
    'stat_type': 'StatType',
    'stat_group': 'StatGroup',
    'season': 'Season',
    'game_date': 'GameDate',
    'team_id': 'TeamID',
    'team_name': 'Team',
    'opponent_id': 'OpponentID',
    'opponent_name': 'Opponent',
    'position': 'Position',
    'gamesPlayed': 'GamesPlayed',
    'games': 'Games',
    'gamesStarted': 'GamesStarted',
    'assists': 'Assists',
    'putOuts': 'PutOuts',
    'errors': 'Errors',
    'chances': 'Chances',
    'fielding': 'Fielding%',
    'doublePlays': 'DoublePlays',
    'triplePlays': 'TriplePlays',
    'throwingErrors': 'ThrowingErrors',
    'rangeFactorPerGame': 'RangeFactor/Game',
    'rangeFactorPer9Inn': 'RangeFactor/9Inn',
    'innings': 'Innings',
    'inningsPitched': 'InningsPitched',
    'catcherERA': 'CatcherERA',
    'flyOuts': 'FlyOuts',
    'groundOuts': 'GroundOuts',
    'airOuts': 'AirOuts',
    'passedBall': 'PassedBall',
    'wins': 'Wins',
    'losses': 'Losses',
    'wildPitches': 'WildPitches',
    'pickoffs': 'Pickoffs',
    'runs': 'Runs',
    'doubles': 'Doubles',
    'triples': 'Triples',
    'homeRuns': 'HomeRuns',
    'strikeOuts': 'Strikeouts',
    'baseOnBalls': 'Walks',
    'intentionalWalks': 'IntentionalWalks',
    'hits': 'Hits',
    'hitByPitch': 'HitByPitch',
    'avg': 'BattingAvg',
    'atBats': 'AtBats',
    'obp': 'OnBase%',
    'slg': 'Slugging%',
    'ops': 'OPS',
    'caughtStealing': 'CaughtStealing',
    'stolenBases': 'StolenBases',
    'stolenBasePercentage': 'Steal%',
    'groundIntoDoublePlay': 'GIDP',
    'groundIntoTriplePlay': 'GITP',
    'numberOfPitches': 'PitchesThrown',
    'plateAppearances': 'PlateAppearances',
    'totalBases': 'TotalBases',
    'rbi': 'RBIs',
    'leftOnBase': 'LOB',
    'sacBunts': 'SacBunts',
    'sacFlies': 'SacFlies',
    'babip': 'BABIP',
    'groundOutsToAirouts': 'GO/AO',
    'catchersInterference': 'CatchersInterference',
    'atBatsPerHomeRun': 'AB/HR',
    'summary': 'Summary',
    'isHome': 'HomeGame',
    'isWin': 'Win',
    'game_id': 'GameID',
    'game_number': 'GameNumber',
    'day_night': 'GameTime',
    'era': 'ERA',
    'game': 'Game',
    'id': 'RowID',
    'firstName': 'FirstName',
    'lastName': 'LastName',
    'birthDate': 'BirthDate',
    'currentAge': 'Age',
    'birthCity': 'BirthCity',
    'birthStateProvince': 'BirthState',
    'birthCountry': 'BirthCountry',
    'height': 'Height',
    'weight': 'Weight',
    'primaryPosition': 'PrimaryPosition',
    'batSide': 'BatSide',
    'pitchHand': 'PitchHand',
    'debutDate': 'DebutDate',
    'active': 'IsActive'
})

# Show the updated DataFrame
player_pre.columns

Index(['fullName', 'mlb_player_id', 'mlb_player_id', 'stat_type', 'stat_group',
       'season', 'game_date', 'team_id', 'team_name', 'opponent_id',
       'opponent_name', 'position', 'gamesPlayed', 'games', 'gamesStarted',
       'assists', 'putOuts', 'errors', 'chances', 'fielding', 'doublePlays',
       'triplePlays', 'throwingErrors', 'rangeFactorPerGame',
       'rangeFactorPer9Inn', 'innings', 'inningsPitched', 'catcherERA',
       'flyOuts', 'groundOuts', 'airOuts', 'passedBall', 'wins', 'losses',
       'wildPitches', 'pickoffs', 'runs', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'avg', 'atBats', 'obp', 'slg', 'ops', 'caughtStealing', 'stolenBases',
       'stolenBasePercentage', 'groundIntoDoublePlay', 'groundIntoTriplePlay',
       'numberOfPitches', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'babip', 'groundOutsToAirouts',
       'catchersInterference', 'at

In [8]:
player_df.head(10)

,Name,PlayerID,PlayerID,StatType,StatGroup,Season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,Assists,PutOuts,Errors,Chances,Fielding%,DoublePlays,TriplePlays,ThrowingErrors,RangeFactor/Game,RangeFactor/9Inn,Innings,InningsPitched,CatcherERA,FlyOuts,GroundOuts,AirOuts,PassedBall,Wins,Losses,WildPitches,Pickoffs,Runs,Doubles,Triples,HomeRuns,Strikeouts,Walks,IntentionalWalks,Hits,HitByPitch,BattingAvg,AtBats,OnBase%,Slugging%,OPS,CaughtStealing,StolenBases,Steal%,GIDP,GITP,PitchesThrown,PlateAppearances,TotalBases,RBIs,LOB,SacBunts,SacFlies,BABIP,GO/AO,CatchersInterference,AB/HR,Summary,HomeGame,Win,GameID,GameNumber,GameTime,ERA,Game,RowID,FirstName,LastName,BirthDate,Age,BirthCity,BirthState,BirthCountry,Height,Weight,PrimaryPosition,BatSide,PitchHand,DebutDate,IsActive
0,Justin Verlander,434378,434378,season,fielding,2024,None,117.0,Houston Astros,NaN,None,NA,17,17,17.0,5.0,4.0,0.0,9.0,1.000,0.0,0.0,0.0,0.53,0.90,90.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,1018,Justin,Verlander,1983-02-20,42,Manakin-Sabot,VA,USA,"6' 5""",240,Pitcher,Right,Right,2005-07-04,1
1,Kenley Jansen,445276,445276,season,hitting,2024,None,111.0,Boston Red Sox,NaN,None,None,54,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,0.0,0.0,NaN,None,None,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,0.0,.000,.000,.000,0.0,0.0,.---,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.---,-.--,0.0,-.--,None,NaN,NaN,NaN,NaN,None,na,na,466,Kenley,Jansen,1987-09-30,37,Willemstad,None,Curacao,"6' 5""",265,Pitcher,Switch,Right,2010-07-24,1
2,Kenley Jansen,445276,445276,season,fielding,2024,None,111.0,Boston Red Sox,NaN,None,NA,54,54,0.0,1.0,1.0,0.0,2.0,1.000,0.0,0.0,0.0,0.04,0.33,54.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,466,Kenley,Jansen,1987-09-30,37,Willemstad,None,Curacao,"6' 5""",265,Pitcher,Switch,Right,2010-07-24,1
3,Jesse Chavez,445926,445926,season,fielding,2024,None,144.0,Atlanta Braves,NaN,None,NA,46,46,0.0,8.0,2.0,1.0,11.0,.909,0.0,0.0,1.0,0.22,1.43,63.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,186,Jesse,Chavez,1983-08-21,41,San Gabriel,CA,USA,"6' 1""",180,Pitcher,Right,Right,2008-08-27,1
4,Charlie Morton,450203,450203,season,fielding,2024,None,144.0,Atlanta Braves,NaN,None,NA,30,30,30.0,21.0,9.0,3.0,33.0,.909,0.0,0.0,3.0,1.00,1.64,165.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,668,Charles,Morton,1983-11-12,41,Flemington,NJ,USA,"6' 5""",215,Pitcher,Right,Right,2008-06-14,1
5,Max Scherzer,453286,453286,season,fielding,2024,None,140.0,Texas Rangers,NaN,None,NA,9,9,9.0,5.0,4.0,1.0,10.0,.900,0.0,0.0,1.0,1.00,1.88,43.1,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,856,Maxwell,Scherzer,1984-07-27,40,St. Louis,MO,USA,"6' 3""",208,Pitcher,Right,Right,2008-04-29,1
6,Martín Maldonado,455117,455117,season,hitting,2024,None,145.0,Chicago White Sox,NaN,None,None,48,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,30.0,41.0,NaN,None,None,NaN,NaN,9.0,3.0,0.0,4.0,51.0,8.0,0.0,16.0,1.0,.119,135.0,.174,.230,.404,0.0,0.0,.---,5.0,NaN,556.0,147.0,31.0,11.0,56.0,3.0,0.0,.150,0.73,0.0,33.75,None,NaN,NaN,NaN,NaN,None,na,na,576,Martín,Maldonado,1986-08-16,38,Naguabo,None,Puerto Rico,"6' 0""",230,Catcher,Right,Right,2

In [9]:
player_df.to_csv('mlb_stats.csv', index=False)

In [77]:
# Step 1: SQL query to join stats and info for all players
query = """
SELECT mlb_player_stats.*, mlb_player_info.*
FROM mlb_player_stats
JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
WHERE mlb_player_stats.team_name = 'Los Angeles Dodgers'
  AND mlb_player_stats.season = 2024;
"""

# Step 2: Load the result into a pandas DataFrame
dodgers_player_pre = pd.read_sql_query(query, conn)

# Step 4: Rename columns for readability
dodgers_df = dodgers_player_pre.rename(columns={
    'fullName': 'Name',
    'mlb_player_id': 'PlayerID',
    'stat_type': 'StatType',
    'stat_group': 'StatGroup',
    'season': 'Season',
    'game_date': 'GameDate',
    'team_id': 'TeamID',
    'team_name': 'Team',
    'opponent_id': 'OpponentID',
    'opponent_name': 'Opponent',
    'position': 'Position',
    'gamesPlayed': 'GamesPlayed',
    'games': 'Games',
    'gamesStarted': 'GamesStarted',
    'assists': 'Assists',
    'putOuts': 'PutOuts',
    'errors': 'Errors',
    'chances': 'Chances',
    'fielding': 'Fielding%',
    'doublePlays': 'DoublePlays',
    'triplePlays': 'TriplePlays',
    'throwingErrors': 'ThrowingErrors',
    'rangeFactorPerGame': 'RangeFactor/Game',
    'rangeFactorPer9Inn': 'RangeFactor/9Inn',
    'innings': 'Innings',
    'inningsPitched': 'InningsPitched',
    'catcherERA': 'CatcherERA',
    'flyOuts': 'FlyOuts',
    'groundOuts': 'GroundOuts',
    'airOuts': 'AirOuts',
    'passedBall': 'PassedBall',
    'wins': 'Wins',
    'losses': 'Losses',
    'wildPitches': 'WildPitches',
    'pickoffs': 'Pickoffs',
    'runs': 'Runs',
    'doubles': 'Doubles',
    'triples': 'Triples',
    'homeRuns': 'HomeRuns',
    'strikeOuts': 'Strikeouts',
    'baseOnBalls': 'Walks',
    'intentionalWalks': 'IntentionalWalks',
    'hits': 'Hits',
    'hitByPitch': 'HitByPitch',
    'avg': 'BattingAvg',
    'atBats': 'AtBats',
    'obp': 'OnBase%',
    'slg': 'Slugging%',
    'ops': 'OPS',
    'caughtStealing': 'CaughtStealing',
    'stolenBases': 'StolenBases',
    'stolenBasePercentage': 'Steal%',
    'groundIntoDoublePlay': 'GIDP',
    'groundIntoTriplePlay': 'GITP',
    'numberOfPitches': 'PitchesThrown',
    'plateAppearances': 'PlateAppearances',
    'totalBases': 'TotalBases',
    'rbi': 'RBIs',
    'leftOnBase': 'LOB',
    'sacBunts': 'SacBunts',
    'sacFlies': 'SacFlies',
    'babip': 'BABIP',
    'groundOutsToAirouts': 'GO/AO',
    'catchersInterference': 'CatchersInterference',
    'atBatsPerHomeRun': 'AB/HR',
    'summary': 'Summary',
    'isHome': 'HomeGame',
    'isWin': 'Win',
    'game_id': 'GameID',
    'game_number': 'GameNumber',
    'day_night': 'GameTime',
    'era': 'ERA',
    'game': 'Game',
    'id': 'RowID',
    'firstName': 'FirstName',
    'lastName': 'LastName',
    'birthDate': 'BirthDate',
    'currentAge': 'Age',
    'birthCity': 'BirthCity',
    'birthStateProvince': 'BirthState',
    'birthCountry': 'BirthCountry',
    'height': 'Height',
    'weight': 'Weight',
    'primaryPosition': 'PrimaryPosition',
    'batSide': 'BatSide',
    'pitchHand': 'PitchHand',
    'debutDate': 'DebutDate',
    'active': 'IsActive'
})

dodgers_df.head(10)

,PlayerID,StatType,StatGroup,Season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,Assists,PutOuts,Errors,Chances,Fielding%,DoublePlays,TriplePlays,ThrowingErrors,RangeFactor/Game,RangeFactor/9Inn,Innings,InningsPitched,CatcherERA,FlyOuts,GroundOuts,AirOuts,PassedBall,Wins,Losses,WildPitches,Pickoffs,Runs,Doubles,Triples,HomeRuns,Strikeouts,Walks,IntentionalWalks,Hits,HitByPitch,BattingAvg,AtBats,OnBase%,Slugging%,OPS,CaughtStealing,StolenBases,Steal%,GIDP,GITP,PitchesThrown,PlateAppearances,TotalBases,RBIs,LOB,SacBunts,SacFlies,BABIP,GO/AO,CatchersInterference,AB/HR,Summary,HomeGame,Win,GameID,GameNumber,GameTime,ERA,Game,RowID,PlayerID,Name,FirstName,LastName,BirthDate,Age,BirthCity,BirthState,BirthCountry,Height,Weight,PrimaryPosition,BatSide,PitchHand,DebutDate,IsActive
0,500743,season,hitting,2024,None,119,Los Angeles Dodgers,NaN,None,None,103,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,106.0,84.0,NaN,None,None,NaN,NaN,41.0,21.0,0.0,6.0,34.0,23.0,0.0,87.0,3.0,.283,307.0,.337,.410,.747,2.0,8.0,.800,8.0,NaN,1211.0,337.0,126.0,36.0,133.0,2.0,2.0,.301,1.26,0.0,51.17,None,NaN,NaN,NaN,NaN,None,na,na,819,500743,Miguel Rojas,Miguel,Rojas,1989-02-24,36,Los Teques,None,Venezuela,"5' 11""",188,Second Base,Right,Right,2014-06-06,1
1,500743,season,fielding,2024,None,119,Los Angeles Dodgers,NaN,None,NA,1,1,0.0,1.0,0.0,0.0,1.0,1.000,0.0,0.0,0.0,1.00,9.00,1.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,819,500743,Miguel Rojas,Miguel,Rojas,1989-02-24,36,Los Teques,None,Venezuela,"5' 11""",188,Second Base,Right,Right,2014-06-06,1
2,500743,season,fielding,2024,None,119,Los Angeles Dodgers,NaN,None,NA,1,1,0.0,0.0,0.0,0.0,0.0,.000,0.0,0.0,0.0,0.00,0.00,1.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,819,500743,Miguel Rojas,Miguel,Rojas,1989-02-24,36,Los Teques,None,Venezuela,"5' 11""",188,Second Base,Right,Right,2014-06-06,1
3,500743,season,fielding,2024,None,119,Los Angeles Dodgers,NaN,None,NA,13,13,11.0,27.0,16.0,0.0,43.0,1.000,6.0,0.0,0.0,3.31,4.07,95.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,819,500743,Miguel Rojas,Miguel,Rojas,1989-02-24,36,Los Teques,None,Venezuela,"5' 11""",188,Second Base,Right,Right,2014-06-06,1
4,500743,season,fielding,2024,None,119,Los Angeles Dodgers,NaN,None,NA,11,11,8.0,11.0,7.0,1.0,19.0,.947,1.0,0.0,1.0,1.64,2.53,64.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,819,500743,Miguel Rojas,Miguel,Rojas,1989-02-24,36,Los Teques,None,Venezuela,"5' 11""",188,Second Base,Right,Right,2014-06-06,1
5,500743,season,fielding,2024,None,119,Los Angeles Dodgers,NaN,None,NA,82,82,67.0,160.0,109.0,2.0,271.0,.993,20.0,0.0,1.0,3.28,4.06,596.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,819,500743,Miguel Rojas,Miguel,Rojas,1989-02-24,36,Los Teques,None,Venezuela,"5' 11""",188,Second Base,Right,Right,2014-06-06,1
6,518489,season,fielding,2024,None,119,Los Angeles Dodgers,NaN,None,NA,29,29,4.0,4.0,4.0,0.0,8.0,1.000,0.0,0.0,0.0,0.28,2.57,28.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,120,518489,Ryan Brasier,Ryan,Brasier,1987-08-26

In [61]:
dodgers_2025 = player_df[(player_df['Team'] == 'Los Angeles Dodgers') & (player_df['Season'] == 2023)]

dodgers_2025

,Name,PlayerID,PlayerID,StatType,StatGroup,Season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,Assists,PutOuts,Errors,Chances,Fielding%,DoublePlays,TriplePlays,ThrowingErrors,RangeFactor/Game,RangeFactor/9Inn,Innings,InningsPitched,CatcherERA,FlyOuts,GroundOuts,AirOuts,PassedBall,Wins,Losses,WildPitches,Pickoffs,Runs,Doubles,Triples,HomeRuns,Strikeouts,Walks,IntentionalWalks,Hits,HitByPitch,BattingAvg,AtBats,OnBase%,Slugging%,OPS,CaughtStealing,StolenBases,Steal%,GIDP,GITP,PitchesThrown,PlateAppearances,TotalBases,RBIs,LOB,SacBunts,SacFlies,BABIP,GO/AO,CatchersInterference,AB/HR,Summary,HomeGame,Win,GameID,GameNumber,GameTime,ERA,Game,RowID,FirstName,LastName,BirthDate,Age,BirthCity,BirthState,BirthCountry,Height,Weight,PrimaryPosition,BatSide,PitchHand,DebutDate,IsActive


In [ ]:
dodgers_2024 = player_df_cleaned[
    (player_df_cleaned['Team'] == 'Los Angeles Dodgers') &
    (player_df_cleaned['season'] == 2024)
]